In [ ]:
#imports

import numpy as np
import sys 
import os
import subprocess
sys.path.append("./Python")
sys.path.append("./MMSP")
sys.path.append("C:/Apps/Cygwin/bin")
from pfutils import *
from python_engine import *
%matplotlib inline
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
np.set_printoptions(threshold=np.inf)

In [ ]:
#initialize the simulation, includes where to save files, the initial step number of zero, and how many steps to simulate per run

path = 'data/dendrites/'
step = 0
steps = 500
nbc = [False, True]
initializePlaneFront(200, 200, path) #parameters: x-size, y-size, path to save files. Plane-front always uses [False, True] for nbc
#initializeSeeds(200, 200, nbc[0], nbc[1], 20, path) #params: x-size, y-size, Neumann Boundary Conditions (Y, X), number of seeds, path to save files
#initializeSeed(200, 200, nbc[0], nbc[1], path) #params: x-size, y-size, Neumann Boundary Conditions (Y, X), path to save files

In [ ]:
#this runs the python engine for the data files selected, then plots the field data as images
# format for .py file params: path to save data, Neumann Boundary Conditions in Y and X dimensions, 
# step to start, steps to simulate, thermal gradient per cell along x axis, change in T per tick

for i in range(1): #number of iterations
    %run -ni ./Python/python_engine {path} {nbc[0]} {nbc[1]} {step} {steps} 0.0 0.0
    plotImages(phi, c, q4, nbc, path, step)


In [ ]:
#this code creates a 3D grid using MMSP
# WARNING - code is proof-of-concept, and may not work on your machine! 
subprocess.Popen(["C:/Users/peters9/phd/ipynb/current/MMSP/initialize.out", "d:3", "f:MMSP/grid", "o:0.8165,0.3333,0.3333,0.3333", "m:0", "l:25,25,25"])

In [ ]:
#this code runs a 3D grid using MMSP
# WARNING - code is proof-of-concept, and may not work on your machine! 
subprocess.Popen(["C:/Users/peters9/phd/ipynb/current/MMSP/serial.out", "MMSP/grid", "40", "10"])

# Anisotropic Dorr Model

This model begins with the following Free Energy functional, identical to that presented in Dorr2010.

$$ F = \int_V (\frac{\epsilon_\phi^2}{2}|\nabla \phi|^2 + f(\phi,c) + f_{ori}(\phi, \textbf{q}) + \frac{\epsilon_q^2}{2}|\nabla \textbf{q}|^2) dV$$ 

In order, these terms are:

* The interfacial energy in phi
* The bulk free energy
* The orientational energy, representing the energy of lattice mismatch
* The interfacial energy in the orientation (q), artificially added to have smooth orientation transitions

To add anisotropic growth, we add a directional component ($\eta$), to the interfacial energy in phi:

$$ F = \int_V (\frac{\epsilon_\phi^2}{2} \eta|\nabla \phi|^2 + f(\phi,c) + f_{ori}(\phi, \textbf{q}) + \frac{\epsilon_q^2}{2}|\nabla \textbf{q}|^2) dV$$ 

We use the following expression to promote cubic symmetry in the PFM:

$$ \eta = 1 - 3\gamma_\epsilon + 4\gamma_\epsilon\frac{\psi_x^4 + \psi_y^4 + \psi_z^4}{|\nabla \phi|^4} $$ 

Here, $\gamma_\epsilon$ represents the magnitude of the anisotropy in the interfacial energy, and all $\psi$ terms are found from rotating the components of $\nabla \phi$ by $\textbf{q}$:

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = (q_1 + q_2\textbf{i} + q_3\textbf{j} + q_4\textbf{k})*(\phi_x\textbf{i} + \phi_y\textbf{j} + \phi_z\textbf{k})*(q_1 - q_2\textbf{i} - q_3\textbf{j} - q_4\textbf{k})$$

It is important to note that, since we are using quaternions, certain multiplications are non-commutative.

$$\textbf{i}\textbf{i} = \textbf{j}\textbf{j} = \textbf{k}\textbf{k} = -1, \textbf{i}\textbf{j} = \textbf{k}, \textbf{j}\textbf{k} = \textbf{i}, \textbf{k}\textbf{i} = \textbf{j}, \textbf{j}\textbf{i} = \textbf{-k}, \textbf{k}\textbf{j} = \textbf{-i}, \textbf{i}\textbf{k} = \textbf{-j}$$

By multiplying out the expression for psi, we may derive equations for them dependent on the various components of $\phi$, and the four quaternion components:

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = (q_1 + q_2\textbf{i} + q_3\textbf{j} + q_4\textbf{k})*(\phi_x\textbf{i} + \phi_y\textbf{j} + \phi_z\textbf{k})*(q_1 - q_2\textbf{i} - q_3\textbf{j} - q_4\textbf{k})$$

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = (q_1\phi_x\textbf{i} + q_1\phi_y\textbf{j} + q_1\phi_z\textbf{k} + q_2\phi_x\textbf{i}\textbf{i} + q_2\phi_y\textbf{i}\textbf{j} + q_2\phi_z\textbf{i}\textbf{k} + q_3\phi_x\textbf{j}\textbf{i} + q_3\phi_y\textbf{j}\textbf{j} + q_3\phi_z\textbf{j}\textbf{k} + q_4\phi_x\textbf{k}\textbf{i} + q_4\phi_y\textbf{k}\textbf{j} + q_4\phi_z\textbf{k}\textbf{k})*(q_1 - q_2\textbf{i} - q_3\textbf{j} - q_4\textbf{k})$$

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = (q_1\phi_x\textbf{i} + q_1\phi_y\textbf{j} + q_1\phi_z\textbf{k} - q_2\phi_x + q_2\phi_y\textbf{k} - q_2\phi_z\textbf{j} - q_3\phi_x\textbf{k} - q_3\phi_y + q_3\phi_z\textbf{i} + q_4\phi_x\textbf{j} - q_4\phi_y\textbf{i} - q_4\phi_z)*(q_1 - q_2\textbf{i} - q_3\textbf{j} - q_4\textbf{k})$$

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = q_1^2\phi_x\textbf{i} + q_1^2\phi_y\textbf{j} + q_1^2\phi_z\textbf{k} - q_1q_2\phi_x + q_1q_2\phi_y\textbf{k} - q_1q_2\phi_z\textbf{j} - q_1q_3\phi_x\textbf{k} - q_1q_3\phi_y + q_1q_3\phi_z\textbf{i} + q_1q_4\phi_x\textbf{j} - q_1q_4\phi_y\textbf{i} - q_1q_4\phi_z - q_1q_2\phi_x\textbf{i}\textbf{i} - q_1q_2\phi_y\textbf{j}\textbf{i} - q_1q_2\phi_z\textbf{k}\textbf{i} + q_2^2\phi_x\textbf{i} - q_2^2\phi_y\textbf{k}\textbf{i} + q_2^2\phi_z\textbf{j}\textbf{i} + q_2q_3\phi_x\textbf{k}\textbf{i} + q_2q_3\phi_y\textbf{i} - q_2q_3\phi_z\textbf{i}\textbf{i} - q_2q_4\phi_x\textbf{j}\textbf{i} + q_2q_4\phi_y\textbf{i}\textbf{i} + q_2q_4\phi_z\textbf{i} - q_1q_3\phi_x\textbf{i}\textbf{j} - q_1q_3\phi_y\textbf{j}\textbf{j} - q_1q_3\phi_z\textbf{k}\textbf{j} + q_2q_3\phi_x\textbf{j} - q_2q_3\phi_y\textbf{k}\textbf{j} + q_2q_3\phi_z\textbf{j}\textbf{j} + q_3^2\phi_x\textbf{k}\textbf{j} + q_3^2\phi_y\textbf{j} - q_3^2\phi_z\textbf{i}\textbf{j} - q_3q_4\phi_x\textbf{j}\textbf{j} + q_3q_4\phi_y\textbf{i}\textbf{j} + q_3q_4\phi_z\textbf{j} - q_1q_4\phi_x\textbf{i}\textbf{k} - q_1q_4\phi_y\textbf{j}\textbf{k} - q_1q_4\phi_z\textbf{k}\textbf{k} + q_2q_4\phi_x\textbf{k} - q_2q_4\phi_y\textbf{k}\textbf{k} + q_2q_4\phi_z\textbf{j}\textbf{k} + q_3q_4\phi_x\textbf{k}\textbf{k} + q_3q_4\phi_y\textbf{k} - q_3q_4\phi_z\textbf{i}\textbf{k} - q_4^2\phi_x\textbf{j}\textbf{k} + q_4^2\phi_y\textbf{i}\textbf{k} + q_4^2\phi_z\textbf{k}$$

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = q_1^2\phi_x\textbf{i} + q_1^2\phi_y\textbf{j} + q_1^2\phi_z\textbf{k} - q_1q_2\phi_x + q_1q_2\phi_y\textbf{k} - q_1q_2\phi_z\textbf{j} - q_1q_3\phi_x\textbf{k} - q_1q_3\phi_y + q_1q_3\phi_z\textbf{i} + q_1q_4\phi_x\textbf{j} - q_1q_4\phi_y\textbf{i} - q_1q_4\phi_z + q_1q_2\phi_x + q_1q_2\phi_y\textbf{k} - q_1q_2\phi_z\textbf{j} + q_2^2\phi_x\textbf{i} - q_2^2\phi_y\textbf{j} - q_2^2\phi_z\textbf{k} + q_2q_3\phi_x\textbf{j} + q_2q_3\phi_y\textbf{i} + q_2q_3\phi_z + q_2q_4\phi_x\textbf{k} - q_2q_4\phi_y + q_2q_4\phi_z\textbf{i} - q_1q_3\phi_x\textbf{k} + q_1q_3\phi_y + q_1q_3\phi_z\textbf{i} + q_2q_3\phi_x\textbf{j} + q_2q_3\phi_y\textbf{i} - q_2q_3\phi_z - q_3^2\phi_x\textbf{i} + q_3^2\phi_y\textbf{j} - q_3^2\phi_z\textbf{k} + q_3q_4\phi_x + q_3q_4\phi_y\textbf{k} + q_3q_4\phi_z\textbf{j} + q_1q_4\phi_x\textbf{j} - q_1q_4\phi_y\textbf{i} + q_1q_4\phi_z + q_2q_4\phi_x\textbf{k} + q_2q_4\phi_y + q_2q_4\phi_z\textbf{i} - q_3q_4\phi_x + q_3q_4\phi_y\textbf{k} + q_3q_4\phi_z\textbf{j} - q_4^2\phi_x\textbf{i} - q_4^2\phi_y\textbf{j} + q_4^2\phi_z\textbf{k}$$

In this expression, every term that does not have a imaginary component (i,j,k) will have a matching, opposite sign term. Therefore, all these terms will cancel out. Then, we group terms based on which component they have:

$$\psi_x\textbf{i} + \psi_y\textbf{j} + \psi_z\textbf{k} = (q_1^2\phi_x + q_1q_3\phi_z - q_1q_4\phi_y + q_2^2\phi_x + q_2q_3\phi_y + q_2q_4\phi_z + q_1q_3\phi_z + q_2q_3\phi_y - q_3^2\phi_x - q_1q_4\phi_y + q_2q_4\phi_z - q_4^2\phi_x)\textbf{i} + (q_1^2\phi_y - q_1q_2\phi_z + q_1q_4\phi_x - q_1q_2\phi_z - q_2^2\phi_y + q_2q_3\phi_x + q_2q_3\phi_x + q_3^2\phi_y + q_3q_4\phi_z + q_1q_4\phi_x + q_3q_4\phi_z - q_4^2\phi_y)\textbf{j} + (q_1^2\phi_z + q_1q_2\phi_y - q_1q_3\phi_x + q_1q_2\phi_y - q_2^2\phi_z + q_2q_4\phi_x - q_1q_3\phi_x - q_3^2\phi_z + q_3q_4\phi_y + q_2q_4\phi_x + q_3q_4\phi_y + q_4^2\phi_z)\textbf{k}$$

From this equation, we can get the final expression for each of the three psi components:

$$\psi_x = (q_1^2 + q_2^2 - q_3^2 - q_4^2)\phi_x + (2q_2q_3 - 2q_1q_4)\phi_y + (2q_2q_4 + 2q_1q_3)\phi_z$$

$$\psi_y = (2q_2q_3 + 2q_1q_4)\phi_x + (q_1^2 - q_2^2 + q_3^2 - q_4^2)\phi_y + (2q_3q_4 - 2q_1q_2)\phi_z$$

$$\psi_z = (2q_2q_4 - 2q_1q_3)\phi_x + (2q_3q_4 + 2q_1q_2)\phi_y + (q_1^2 - q_2^2 - q_3^2 + q_4^2)\phi_z$$

In 2D, the only meaningful rotation is within the xy plane, so we require that $q_2 = q_3 = 0$ for the 2D model. Additionally, $\phi_z = 0$. As a consequence, the expressions for $\psi$ simplify in the following manner:

$$\psi_x = (q_1^2 - q_4^2)\phi_x - 2q_1q_4\phi_y$$

$$\psi_y = 2q_1q_4\phi_x + (q_1^2 - q_4^2)\phi_y$$

$$\psi_z = 0$$

Returning to the function equation, the bulk free energy, and the orientational mismatch energy are represented by the following equations:

$$ f(\phi, c) = (1-c)(W^Ag(\phi)T + (1-\frac{T}{T_m^A})(e_s^A(T_m^A) - C^AT_m^A + m(\phi)L^A) - C^ATln\frac{T}{T_m^A} + \frac{RT}{v_m}ln(1-c)) + c(W^Bg(\phi)T + (1-\frac{T}{T_m^B})(e_s^B(T_m^B) - C^BT_m^B + m(\phi)L^B) - C^BTln\frac{T}{T_m^B} + \frac{RT}{v_m}ln(c))$$

$$ f_{ori}(\phi, \textbf{q}) = 2HTp(\phi)|\nabla \textbf{q}|$$

These expressions are dependent on several material properties, derived properties, and physical parameters, as well as the three interpolating functions, $g(\phi)$, $m(\phi)$, and $p(\phi)$. Below, descriptions and/or equations are given where appropriate:

* $g(\phi) = \phi^2(1-\phi)^2$
* $m(\phi) = 1 - \phi^3(10-15\phi+6\phi^2)$
* $p(\phi) = \phi^2$
* $T$: Temperature (in Kelvin)
* $T_m^A$, $T_m^B$: Melting temperature of pure material A and B
* $e_s^A(T_X)$, $e_s^B(T_X)$: Internal energies of pure A and B, at some temperature $T_X$
* $C^A$, $C^B$: Heat capacities of pure A and B
* $L^A$, $L^B$: Latent heat of pure A and B
* $R$: Gas constant
* $v_m$: molar volume
* $W^A$: Well height of A, $= \frac{3\sigma^A}{\sqrt{2}T_m^A\delta^A}$
* $W^B$: Well height of B, $= \frac{3\sigma^B}{\sqrt{2}T_m^B\delta^B}$
* $\sigma^A$, $\sigma^B$: Surface energy of pure A and B
* $\delta^A$, $\delta^B$: Interfacial thickness of A and B

We also define the interfacial energy coefficient, $\epsilon_\phi$, using these terms as well:

* $\epsilon_\phi^2 = \frac{6\sqrt{2}\sigma^A\delta^A}{T_m^A} = \frac{6\sqrt{2}\sigma^B\delta^B}{T_m^B}$

In this simulation, A is pure Nickel, and B is pure Copper, with the simulation being treated as an ideal solution. Many of these expressions are taken from Warren1995.

